In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
datasets_path = "../datasets_for_ml/"

reservas_futuras = pd.read_csv(datasets_path+'reservas_futuras.csv')
transacoes = pd.read_csv(datasets_path+'transacoes.csv')
cancelados = pd.read_csv(datasets_path+'cancelados.csv')
faltantes = pd.read_csv(datasets_path+'faltantes.csv')

transacoes['Date'] = pd.to_datetime(transacoes['Date'])
cancelados['Booking Date'] = pd.to_datetime(cancelados['Booking Date'])
cancelados['Cancel Date'] = pd.to_datetime(cancelados['Cancel Date'])
faltantes['Date'] = pd.to_datetime(faltantes['Date'])
reservas_futuras['Date'] = pd.to_datetime(reservas_futuras['Date'])

Vimos na análise dos dados que há interseções entre as tabelas de transações, cancelados e faltantes. Entretanto, suspeitamos que essas tabelas deveriam ser mutuamente exclusivas, de tal forma que não deveria haver interseções entre elas.

Vamos tomar a seguinte estratégia para lidar com esse problema:

* Todas as interseções de cancelados e faltantes com transações serão removidos, deixando as instâncias somente na tabela transações. Isso pois se o cliente está na tabela de transações significa que ele recebeu o serviço, portanto, não cancelou e nem faltou.

* Os clientes que estão na interseção de cancelados e faltantes serão removidos da tabela de faltantes e mantidos na tabela de cancelados. Se o cliente está na lista de cancelados, significa que ele cancelou e, portanto, não faltou.

Criando flags para auxiliar na remoção das instâncias

In [3]:
# Apenas para controle
transacoes.shape , cancelados.shape , faltantes.shape

((1862, 13), (240, 7), (59, 4))

In [4]:
cancelados['to_remove'] = cancelados.index
faltantes['to_remove'] = faltantes.index

In [5]:
remove = pd.merge(left = cancelados[['Booking Date','Code','to_remove']], 
               right = transacoes[['Date','Client']],
               left_on = ['Code','Booking Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True).drop_duplicates()['to_remove']


print(f"Quantidade de interseções entre cancelados e transacoes: {len(remove)}")

cancelados = cancelados.drop(remove)



Quantidade de interseções entre cancelados e transacoes: 56


In [6]:
remove = pd.merge(left = faltantes[['Date','Code','to_remove']], 
               right = transacoes[['Date','Client']],
               left_on = ['Code','Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True).drop_duplicates()['to_remove']


print(f"Quantidade de interseções entre faltantes e transacoes: {len(remove)}")
faltantes = faltantes.drop(remove)

Quantidade de interseções entre faltantes e transacoes: 1


In [7]:
remove = pd.merge(left = cancelados[['Booking Date','Code']], 
               right = faltantes[['Date','Code','to_remove']],
               left_on = ['Code','Booking Date'], 
               right_on=['Code','Date'],
               how='inner',
               indicator=True).drop_duplicates()['to_remove']


print(f"Quantidade de interseções entre cancelados e transacoes: {len(remove)}")
faltantes = faltantes.drop(remove)

Quantidade de interseções entre cancelados e transacoes: 4


In [8]:
cancelados = cancelados.drop('to_remove',axis=1)
faltantes = faltantes.drop('to_remove',axis=1)

In [9]:
# Apenas para controle
transacoes.shape , cancelados.shape , faltantes.shape

((1862, 13), (184, 7), (54, 4))

Nota-se que não há mais interseções entre as tabelas

In [10]:
print('Transações vs cancelados')
display(pd.merge(left = cancelados, 
               right = transacoes,
               left_on = ['Code','Booking Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True))

print("-"*100,'\nTransações vs faltantes')
display(pd.merge(left = faltantes, 
               right = transacoes,
               left_on = ['Code','Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True))

print("-"*100,'\nFaltantes vs cancelados')
display(pd.merge(left = cancelados, 
               right = faltantes,
               left_on = ['Code','Booking Date'], 
               right_on=['Code','Date'],
               how='inner',
               indicator=True))

Transações vs cancelados


,Cancel Date,Code,Service,Staff_x,Booking Date,Canceled By,Days,Receipt,Date,Description,...,Staff_y,Quantity,Amount,GST,PST,mes_nome,mes_n,dia_nome,dia_n,_merge


---------------------------------------------------------------------------------------------------- 
Transações vs faltantes


,Date,Code,Service,Staff_x,Receipt,Description,Client,Staff_y,Quantity,Amount,GST,PST,mes_nome,mes_n,dia_nome,dia_n,_merge


---------------------------------------------------------------------------------------------------- 
Faltantes vs cancelados


,Cancel Date,Code,Service_x,Staff_x,Booking Date,Canceled By,Days,Date,Service_y,Staff_y,_merge


## **Construção do dataset para treinamento, teste e validação**

O objetivo do projeto consiste em desenvolver um classificador que seja capaz de identificar os clientes que não sigam as políticas do salão. Portanto, as únicas informações que teremos acesso para realizar essa predição será o histórico do cliente e as informações passadas por ele no momento da reserva.

Logo, o histórico do cliente será de muita importância para o nosso desenvolvimento. Portanto, não podemos simplesmente elaborar um dataset de treino considerando todas as instâncias das tabelas, uma vez que existe uma relação temporal nos dados mesmo que o problema não se configure com o de uma série temporal.

Dito isso, vamos seguir da seguinte maneira:

> 1. Vamos separar os primeiros quatro meses (Março a Junho) para elaborar o dataset de treino. Os meses subsequentes serão utilizados para teste.

> 2. Com a tabela de faltantes vamos criar uma flag para realizar a contagem de quantas vezes um dado cliente faltou durante os 3 meses de referência. Podemos obter também o staff e serviço mais frequente durante esse histórico de faltas. 

> 3. Com a tabela de cancelados podemos gerar uma flag informando quantas vezes um determinado cliente cancelou. Podemos obter a média/mediana da diferença entre as datas de cancelamento e reserva, o staff mais frequente e o serviço mais frequente.

> 4. Com a tabela transações conseguimos obter as métricas de recência, frequência e valor, o staff e dia mais frequentes além da quantidade de serviços prestado por dia.


O target será composto por um valor binário, o valor zero corresponderá aos clientes que seguiu a política do salão e o valor um corresponderá aos clientes que não seguiram a política do salão.

Desse modo, será atribuído o target igual a 1 para todas as instâncias que  tiverem o número de faltas acima de 0 e/ou as instâncias cujas diferenças entre a data de reserva e de cancelamento são menores que dois dias. Serão atribuídas o valor 0 para as demais instâncias.


Como vimos no EDA, a tabela reservas futuras corresponde justamente às reservas realizadas pelos cientes. A princípio não pretendo utilizar essa tabela, dado que o treino e o teste eu consigo das demais tabelas. Entretanto, o que receberíamos em produção seria justamente a tabela de reservas futuras. Com essa tabela conseguimos rastrear o histórico do cliente e efetuar a predição.

Um ponto de observação é importante. Clientes que nunca foram ao salão não apresentam histórico, desse modo, precisaríamos lidar com essa situação em produção. Ao meu ver, o mais prudente seria não realizar a estimativa para esses novos clientes, uma vez que não há o histórico deles.

Observação: No EDA concluímos que o nosso intervalo de tempo compreende 136 dias ou pouco mais de 4 meses. Entretanto, esses dias estão distribuídos ao longo de Março a Julho. Esse pequeno detalhe pode gerar algumas dúvidas.

In [11]:
transacoes_train = transacoes[transacoes['Date'].apply(lambda x: x.month <= 6)]
transacoes_test = transacoes[transacoes['Date'].apply(lambda x: x.month > 6)]

cancelados_train = cancelados[cancelados['Booking Date'].apply(lambda x: x.month <= 6)]
cancelados_test =  cancelados[cancelados['Booking Date'].apply(lambda x: x.month >6)]

faltantes_train = faltantes[faltantes['Date'].apply(lambda x: x.month <= 6)]
faltantes_test =  faltantes[faltantes['Date'].apply(lambda x: x.month > 6)]

train = pd.DataFrame()
test = pd.DataFrame()


In [12]:
train['cliente'] = transacoes_train['Client'].unique()

test['cliente'] = transacoes_test['Client'].unique()

In [13]:
train.shape,test.shape

((650, 1), (321, 1))

Conferindo a seguir os meses para os datasets de treino  e teste

In [14]:
transacoes_train['mes_nome'].unique()

array(['March', 'April', 'June', 'May'], dtype=object)

In [15]:
transacoes_test['mes_nome'].unique()

array(['July'], dtype=object)

Vamos iniciar da tabela mais simples para a tabela mais complicada. Vamos começar por faltantes

1. Calcular a quantidade de vezes que houve falta por cliente

2. Staff mais frequente

3. Serviço mais frequente

In [16]:
qt_faltas = faltantes_train.groupby('Code').size().to_frame('qte_faltas').reset_index()

train = pd.merge(left = train,right = qt_faltas, left_on='cliente',right_on='Code',how='left')
train = train.fillna(0)
train = train.drop('Code',axis=1)


In [17]:
service_moda = faltantes_train.groupby('Code').apply(lambda x: x['Service'].mode()[0]).to_frame('moda_servico_faltante').reset_index()

train = pd.merge(left = train,right = service_moda, left_on='cliente',right_on='Code',how='left')
train = train.drop('Code',axis=1)

In [18]:
staff_moda = faltantes_train.groupby('Code').apply(lambda x: x['Staff'].mode()[0]).to_frame('moda_staff_faltante').reset_index()

train = pd.merge(left = train,right = staff_moda, left_on='cliente',right_on='Code',how='left')
train = train.drop('Code',axis=1)

Vamos para os cancelados

1. Calcular o serviço mais frequente

2. Staff mais frequente

3. Staff que cancelou mais frequente

4. A média da antecedência de cancelamento

5. Quantidade de vezes que cancelou

In [19]:
aux = cancelados_train.groupby('Code').apply(lambda x: x['Service'].mode()[0]).to_frame('moda_servico_cancelado').reset_index()


train = pd.merge(left = train,right = aux, left_on='cliente',right_on='Code',how='left')
train = train.drop('Code',axis=1)


In [20]:
aux = cancelados_train.groupby('Code').apply(lambda x: x['Staff'].mode()[0]).to_frame('moda_staff_cancelado').reset_index()


train = pd.merge(left = train,right = aux, left_on='cliente',right_on='Code',how='left')
train = train.drop('Code',axis=1)


In [21]:
aux = cancelados_train.groupby('Code').apply(lambda x: x['Service'].mode()[0]).to_frame('moda_staff_cancelador_cancelado').reset_index()


train = pd.merge(left = train,right = aux, left_on='cliente',right_on='Code',how='left')
train = train.drop('Code',axis=1)


In [22]:
aux = cancelados_train.groupby('Code')['Days'].mean().to_frame('antecedencia').reset_index()

train = pd.merge(left = train,right = aux, left_on='cliente',right_on='Code',how='left')
train = train.drop('Code',axis=1)

In [23]:
aux = cancelados_train.groupby('Code').size().to_frame('qte_cancelamento').reset_index()

train = pd.merge(left = train,right = aux, left_on='cliente',right_on='Code',how='left')
train = train.drop('Code',axis=1)
train['qte_cancelamento'] = train['qte_cancelamento'].fillna(0)

Vamos para a tabela transacoes

1. Calcular a média da quantidade de serviços por dia (receipt)

2. Calcular o Staff mais frequente

3. Calcular recência

4. Calcular frequência

5. Calcular valor

6. Calcular o dia_nome mais frequente

In [24]:
transacoes_train.head()

,Receipt,Date,Description,Client,Staff,Quantity,Amount,GST,PST,mes_nome,mes_n,dia_nome,dia_n
0,10,2018-03-15,Women's hair cut,KERT01,JJ,1,82.0,4.10,6.56,March,3,Thursday,3
1,100,2018-04-05,Women's hair cut,COOM01,SINEAD,1,70.0,3.50,5.60,April,4,Thursday,3
2,1000,2018-06-09,Consultation,PEDM01,JJ,1,0.0,0.00,0.00,June,6,Saturday,5
3,1001,2018-06-09,Women's hair cut,BAIS01,KELLY,1,55.0,2.75,4.40,June,6,Saturday,5
4,1003,2018-06-09,Women's hair cut,FRAL01,KELLY,1,65.0,3.25,5.20,June,6,Saturday,5


In [25]:
aux = transacoes_train.groupby(['Client','Date'])['Receipt'].size().groupby('Client').mean().to_frame('qte_servicos_por_dia').reset_index()

train = pd.merge(left = train,right = aux, left_on='cliente',right_on='Client',how='left')
train = train.drop('Client',axis=1)

In [26]:
aux = transacoes_train.groupby('Client').apply(lambda x: x['Staff'].mode()[0]).to_frame('moda_staff_prestou_servico').reset_index()


train = pd.merge(left = train,right = aux, left_on='cliente',right_on='Client',how='left')
train = train.drop('Client',axis=1)


In [27]:
aux = transacoes_train.groupby('Client').apply(lambda x: x['dia_nome'].mode()[0]).to_frame('moda_dia').reset_index()


train = pd.merge(left = train,right = aux, left_on='cliente',right_on='Client',how='left')
train = train.drop('Client',axis=1)


Agora resta calcular recência, frequência e valor